In [1]:
# Libraries
import pandas as pd
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import sklearn.metrics as sm
import math

# Data
appliances = pd.read_csv('https://byui-cse.github.io/cse450-course/ice/energy/data/energy-training.csv')

In [ ]:
# Data dictionary

# Year, year the reading was taken
# Month, month the reading was taken
# Day, day the reading was taken
# Hours, hour of the day in 0-23 hour format that the reading was taken
# Minutes, beginning of the 10 minute interval during which the reading was * taken (0-50, where 0 means minute 0 through 9)
# Appliances, energy use in Wh
# lights, energy use of light fixtures in the house in Wh
# T1, Temperature in kitchen area, in Celsius
# RH_1, Humidity in kitchen area, in %
# T2, Temperature in living room area, in Celsius
# RH_2, Humidity in living room area, in %
# T3, Temperature in laundry room area
# RH_3, Humidity in laundry room area, in %
# T4, Temperature in office room, in Celsius
# RH_4, Humidity in office room, in %
# T5, Temperature in bathroom, in Celsius
# RH_5, Humidity in bathroom, in %
# T6, Temperature outside the building (north side), in Celsius
# RH_6, Humidity outside the building (north side), in %
# T7, Temperature in ironing room , in Celsius
# RH_7, Humidity in ironing room, in %
# T8, Temperature in teenager room 2, in Celsius
# RH_8, Humidity in teenager room 2, in %
# T9, Temperature in parents room, in Celsius
# RH_9, Humidity in parents room, in %
# To, Temperature outside (from nearest weather station), in Celsius
# Pressure (from nearest weather station), in mm Hg
# RH_out, Humidity outside (from nearest weather station), in %
# Wind speed (from nearest weather station), in m/s
# Visibility (from nearest weather station), in km
# Tdewpoint (from nearest weather station), °C
# rv1, mystery variable 1
# rv2, mystery variable 2

In [ ]:
# Check whata dataset looks like
appliances.head()

,Year,Month,Day,Hours,Minutes,Time-since-start,Appliances,lights,T1,RH_1,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016,5,10,14,50,17267,60,0,25.20,47.126667,...,24.100000,47.525714,18.800,750.150000,78.000000,4.500000,40.000000,14.80,41.358725,41.358725
1,2016,5,4,21,50,16445,60,0,23.39,33.790000,...,20.390000,34.463333,10.900,764.283333,57.333333,3.000000,40.000000,2.77,9.024852,9.024852
2,2016,3,16,20,20,9380,140,20,22.60,37.360000,...,19.600000,36.200000,5.270,765.200000,62.000000,5.666667,23.333333,-1.50,49.291757,49.291757
3,2016,3,19,2,30,9705,50,0,21.79,36.760000,...,19.700000,39.560000,4.700,762.300000,92.000000,3.500000,62.500000,3.50,0.102888,0.102888
4,2016,3,11,5,40,8572,50,0,20.50,36.200000,...,18.166667,41.090000,-0.767,763.600000,93.666667,1.333333,33.000000,-1.70,8.882974,8.882974


In [ ]:
# Check data types
appliances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13900 entries, 0 to 13899
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              13900 non-null  int64  
 1   Month             13900 non-null  int64  
 2   Day               13900 non-null  int64  
 3   Hours             13900 non-null  int64  
 4   Minutes           13900 non-null  int64  
 5   Time-since-start  13900 non-null  int64  
 6   Appliances        13900 non-null  int64  
 7   lights            13900 non-null  int64  
 8   T1                13900 non-null  float64
 9   RH_1              13900 non-null  float64
 10  T2                13900 non-null  float64
 11  RH_2              13900 non-null  float64
 12  T3                13900 non-null  float64
 13  RH_3              13900 non-null  float64
 14  T4                13900 non-null  float64
 15  RH_4              13900 non-null  float64
 16  T5                13900 non-null  float6

In [ ]:
# Check amount of NAs
appliances.isna().sum()

Year                0
Month               0
Day                 0
Hours               0
Minutes             0
Time-since-start    0
Appliances          0
lights              0
T1                  0
RH_1                0
T2                  0
RH_2                0
T3                  0
RH_3                0
T4                  0
RH_4                0
T5                  0
RH_5                0
T6                  0
RH_6                0
T7                  0
RH_7                0
T8                  0
RH_8                0
T9                  0
RH_9                0
T_out               0
Press_mm_hg         0
RH_out              0
Windspeed           0
Visibility          0
Tdewpoint           0
rv1                 0
rv2                 0
dtype: int64

In [ ]:
# Conversion functions
def CelsiusToFahrenheit(x):
  fahrenheit = x * (9/5) + 32
  return fahrenheit

def KilometerToMiles(x):
  miles = x / 1.609

In [2]:
# Combine year, month, day to datetime
appliances['Date'] = pd.to_datetime(appliances.Year.astype(str) + '/' + appliances.Month.astype(str) + '/' + appliances.Day.astype(str))

appliances['Date'].head()

0   2016-05-10
1   2016-05-04
2   2016-03-16
3   2016-03-19
4   2016-03-11
Name: Date, dtype: datetime64[ns]

In [3]:
# Get day name from date
appliances['DayName'] = appliances['Date'].dt.day_name()

appliances['DayName'].head()

0      Tuesday
1    Wednesday
2    Wednesday
3     Saturday
4       Friday
Name: DayName, dtype: object

In [4]:
# Check if day is weekend or not
appliances['IsWeekend'] = appliances['DayName'].isin(['Saturday', 'Sunday']).astype(int)

appliances['IsWeekend'].head()

0    0
1    0
2    0
3    1
4    0
Name: IsWeekend, dtype: int64

In [5]:
# Train, test, split
X = appliances.drop(['Date', 'DayName', 'Appliances'], axis = 1)
y = appliances['Appliances']

X_test, X_train, y_test, y_train = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [12]:
model = XGBRegressor(learning_rate = 0.2, max_depth = 2, subsample = 0.9)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [13]:
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, predictions), 2)) 
print("Mean squared error =", round(math.sqrt(sm.mean_squared_error(y_test, predictions)), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, predictions), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, predictions), 2)) 
print("R2 score =", round(sm.r2_score(y_test, predictions), 6))

Mean absolute error = 46.69
Mean squared error = 91.33
Median absolute error = 22.66
Explain variance score = 0.23
R2 score = 0.224087


In [15]:
# Holdout
holdout = pd.read_csv('https://byui-cse.github.io/cse450-course/ice/energy/data/energy-holdout.csv')

In [ ]:
holdout.head()

,Year,Month,Day,Hours,Minutes,Time-since-start,lights,T1,RH_1,T2,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016,1,31,4,20,2804,0,20.600000,42.200000,19.600,...,16.79,49.500000,4.40,754.900000,90.333333,4.000000,40.000000,2.93,19.438111,19.438111
1,2016,2,15,6,0,4974,0,19.700000,41.060000,17.890,...,17.79,44.290000,2.50,752.900000,87.000000,6.000000,40.000000,0.50,35.047871,35.047871
2,2016,3,6,15,20,7910,0,20.600000,37.826667,19.500,...,17.79,37.590000,3.47,747.266667,91.333333,5.333333,28.000000,2.20,21.252116,21.252116
3,2016,5,16,18,40,18154,0,23.533333,40.193333,22.600,...,22.79,37.795714,14.30,760.300000,46.000000,2.333333,40.000000,2.70,19.834068,19.834068
4,2016,5,12,22,10,17599,0,25.390000,49.163333,24.315,...,24.06,48.036000,16.50,745.083333,78.666667,1.000000,26.666667,12.70,5.429815,5.429815


In [25]:
# Combine year, month, day to datetime
holdout['Date'] = pd.to_datetime(holdout.Year.astype(str) + '/' + holdout.Month.astype(str) + '/' + holdout.Day.astype(str))

# Get day name from date
holdout['DayName'] = holdout['Date'].dt.day_name()

# Check if day is weekend or not
holdout['IsWeekend'] = holdout['DayName'].isin(['Saturday', 'Sunday']).astype(int)

holdout.head()

,Year,Month,Day,Hours,Minutes,Time-since-start,lights,T1,RH_1,T2,...,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,IsWeekend,Date,DayName
0,2016,1,31,4,20,2804,0,20.600000,42.200000,19.600,...,754.900000,90.333333,4.000000,40.000000,2.93,19.438111,19.438111,1,2016-01-31,Sunday
1,2016,2,15,6,0,4974,0,19.700000,41.060000,17.890,...,752.900000,87.000000,6.000000,40.000000,0.50,35.047871,35.047871,0,2016-02-15,Monday
2,2016,3,6,15,20,7910,0,20.600000,37.826667,19.500,...,747.266667,91.333333,5.333333,28.000000,2.20,21.252116,21.252116,1,2016-03-06,Sunday
3,2016,5,16,18,40,18154,0,23.533333,40.193333,22.600,...,760.300000,46.000000,2.333333,40.000000,2.70,19.834068,19.834068,0,2016-05-16,Monday
4,2016,5,12,22,10,17599,0,25.390000,49.163333,24.315,...,745.083333,78.666667,1.000000,26.666667,12.70,5.429815,5.429815,0,2016-05-12,Thursday


In [26]:
# Make sure holdout will match with model
holdout = holdout.drop(['Date', 'DayName'], axis = 1)

In [27]:
holdoutPredict = model.predict(holdout)

In [28]:
pd.DataFrame(holdoutPredict).to_csv('devin-predictions.csv', index = False)